In [ ]:
!pip install qiskit
!pip install qiskit-machine-learning

In [14]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
from qiskit.quantum_info import SparsePauliOp
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit.utils import algorithm_globals


In [15]:
n_qubits = 4

In [16]:
def layer(W):
    """Applies a layer of arbitrary rotations and circular entanglements to the variational circuit

    Args:
        W (np.ndarray): rotation parameters for the layer
    """
    qc = QuantumCircuit(n_qubits)

    for i in range(n_qubits):
        qc.u(W[i*3], W[i*3 + 1], W[i*3 + 2], i)

    for i in range(n_qubits-1):
        qc.cnot(i, i+1)

    if n_qubits > 2:
        qc.cnot(n_qubits-1, 0)

    return qc

In Qiskit, there is not basis state preparation circuit and anyway we need to work with `Parameter` objects to use the `qiskit-machine-learning` tools. So we just encode $R_X$ rotation, pass the parameters and multiply by $\pi$.

In [23]:
def statepreparation(x):
    """Prepares the binary state fed to the vqc

    Args:
        x (List): list of 0s and 1s corresponding to the basis state
    """
    qc = QuantumCircuit(n_qubits)
    
    for i, x_i in enumerate(x):
        qc.rx(x_i * np.pi, i)

    return qc

In [24]:
n_layers = 2

weight_params = ParameterVector(name='W', length=3 * n_qubits * n_layers)
input_params = ParameterVector(name='x', length=n_qubits)

In [25]:
# prepare the quantum circuit
qc = QuantumCircuit(n_qubits)
qc = qc.compose(statepreparation(input_params))
for l in range(n_layers):
    qc = qc.compose(layer(weight_params[3 * n_qubits * l: 3 * n_qubits * (l+1)]))


In [26]:
qc.draw()

┌────────────┐ ┌───────────────────┐                                   »
q_0: ┤ Rx(π*x[0]) ├─┤ U(W[0],W[1],W[2]) ├───■───────────────────────────────»
     ├────────────┤ ├───────────────────┤ ┌─┴─┐     ┌──────────────────────┐»
q_1: ┤ Rx(π*x[1]) ├─┤ U(W[3],W[4],W[5]) ├─┤ X ├──■──┤ U(W[15],W[16],W[17]) ├»
     ├────────────┤ ├───────────────────┤ └───┘┌─┴─┐└──────────────────────┘»
q_2: ┤ Rx(π*x[2]) ├─┤ U(W[6],W[7],W[8]) ├──────┤ X ├───────────■────────────»
     ├────────────┤┌┴───────────────────┴┐     └───┘         ┌─┴─┐          »
q_3: ┤ Rx(π*x[3]) ├┤ U(W[9],W[10],W[11]) ├───────────────────┤ X ├──────────»
     └────────────┘└─────────────────────┘                   └───┘          »
«     ┌───┐┌──────────────────────┐               ┌───┐
«q_0: ┤ X ├┤ U(W[12],W[13],W[14]) ├──■────────────┤ X ├
«     └─┬─┘└──────────────────────┘┌─┴─┐          └─┬─┘
«q_1: ──┼──────────────────────────┤ X ├──■─────────┼──
«       │  ┌──────────────────────┐└───┘┌─┴─┐       │  
«q_2: ──┼──┤ U(W[18],W[19],W[20]) ├─────┤ X ├──■────┼──
«       │  ├──────────────────────┤     └───┘┌─┴─┐  │  
«q_3: ──■──┤ U(W[21],W[22],W[23]) ├──────────┤ X ├──■──
«          └──────────────────────┘          └───┘

In [27]:
# define an observable
observable = SparsePauliOp.from_list([("ZIII", 1)])

In [28]:
# define the qnn
qnn = EstimatorQNN(circuit=qc, observables= observable, input_params=input_params, weight_params=weight_params)

In [30]:
input_trial = [0, 1, 0, 0]
weights_trial = algorithm_globals.random.random(qnn.num_weights)
qnn.forward(input_trial, weights_trial)

array([[-0.14887673]])